In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from emnist import extract_training_samples, extract_test_samples

In [2]:
x_train, y_train = extract_training_samples("letters")
x_test, y_test = extract_test_samples('letters')
img_side = 28
input_shape = (img_side, img_side, 1)

In [3]:
y_train = y_train - 1
y_test = y_test - 1

unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

In [4]:
x_train = x_train.reshape(-1, img_side, img_side, 1)
x_test = x_test.reshape(-1, img_side, img_side, 1)

ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((124800, 28, 28, 1), (124800,), (20800, 28, 28, 1), (20800,))

In [5]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(x_train.shape[0])
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [6]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

model = Sequential([
    Conv2D(16, (3, 3), padding='same', input_shape=input_shape, activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(32, (3, 3), padding='same', activation= 'relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), padding='same', activation= 'relu'),
    MaxPooling2D(pool_size =(2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax'),
])

In [8]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    verbose=1,
    patience=10,
    restore_best_weights=True)

model.fit(
    ds_train,
    epochs=200,
    verbose=1,
    validation_data=ds_test,
    callbacks=[es]
)


Epoch 1/200
975/975 [==============================] - 28s 29ms/step - loss: 0.5756 - accuracy: 0.8224 - val_loss: 0.3062 - val_accuracy: 0.9013
Epoch 2/200
975/975 [==============================] - 30s 31ms/step - loss: 0.2570 - accuracy: 0.9152 - val_loss: 0.2465 - val_accuracy: 0.9194
Epoch 3/200
975/975 [==============================] - 31s 32ms/step - loss: 0.2091 - accuracy: 0.9299 - val_loss: 0.2222 - val_accuracy: 0.9259
Epoch 4/200
975/975 [==============================] - 30s 30ms/step - loss: 0.1862 - accuracy: 0.9362 - val_loss: 0.2089 - val_accuracy: 0.9320
Epoch 5/200
975/975 [==============================] - 29s 29ms/step - loss: 0.1662 - accuracy: 0.9423 - val_loss: 0.1907 - val_accuracy: 0.9365
Epoch 6/200
975/975 [==============================] - 29s 30ms/step - loss: 0.1540 - accuracy: 0.9460 - val_loss: 0.1984 - val_accuracy: 0.9324
Epoch 7/200
975/975 [==============================] - 30s 31ms/step - loss: 0.1436 - accuracy: 0.9493 - val_loss: 0.1877 - val_ac

In [9]:
model.evaluate(ds_test)

163/163 [==============================] - 1s 7ms/step - loss: 0.1834 - accuracy: 0.9417


[0.1834419220685959, 0.9416826963424683]

In [10]:
model.save('mnist_cnn_model.h5')